In [1]:
import pandas as pd
from pymongo import MongoClient
from bson.objectid import ObjectId
from datetime import timedelta
from scipy.stats import wilcoxon
import numpy as np
import datetime
import re
client = MongoClient()
db = client.smartshark_2_2
vcs_systems = pd.DataFrame(list(db.vcs_system.find()))
travisBuilds = pd.DataFrame(list(db.travis_build.find()))
tags = pd.DataFrame(list(db.tag.find()))
issues = pd.DataFrame(list(db.issue.find()))

In [2]:
# IC
projetosComBuilds = travisBuilds['vcs_system_id'].unique()

# Tags of project with IC
idsVcsComBuilds = vcs_systems.query('_id.isin(@projetosComBuilds)').drop_duplicates().dropna()._id
tagsDosProjComBuild = tags.query('vcs_system_id.isin(@idsVcsComBuilds)').drop_duplicates().dropna()

#Release Tags
tagsDeRelase = tagsDosProjComBuild[
    tagsDosProjComBuild['name']
    .str.contains('rc|RC|candidate|CANDIDATE|beta|BETA|alpha|ALPHA')==False].drop_duplicates().dropna()

# Valid Builds of Release Tags
commitsDeRelease = tagsDeRelase.commit_id.unique()
buildsIntegrados = travisBuilds.query('commit_id.isin(@commitsDeRelease)')
stateIndesejados = ['failed', 'canceled']
BuildsValidos = buildsIntegrados.query('~state.isin(@stateIndesejados)')

#Projects with valid builds for their releases
vcsComEntrega = BuildsValidos.vcs_system_id.unique()

# Aggregate builds with the same commit
BuildsValidosPorCommit = BuildsValidos[['_id', 'vcs_system_id', 'commit_id', 'number', 'state', 'started_at', 'finished_at']].groupby(['commit_id'], as_index=False, sort=False).max()

# get the projects with > 2 valid builds for an unique commit
vcsComCD = BuildsValidosPorCommit[['vcs_system_id','_id']].groupby(['vcs_system_id'], as_index=False).agg('count').sort_values(by=['_id'], ascending=False)
vcsComCD = vcsComCD.query('_id > 2')

In [3]:
#Select the projects with CD
vcsList= []
for index, row in vcsComCD.iterrows():
    vcsList.append(row.vcs_system_id)
    
issue_system = pd.DataFrame(list(db.issue_system.find()))
projects = pd.DataFrame(list(db.project.find()))
listaDosProjetosComCD = [] 
issues_system_ids = []
for vcs in vcsList:
    # Buscar informações sobre o projeto
    project_id = vcs_systems.query('_id == @vcs').iloc[0,2] 

    #nome do projeto
    name = projects.query('_id == @project_id').iloc[0,1]     

    # Informações sobre o issue system
    issues_system_id = issue_system.query('project_id == @project_id').iloc[0,0]
    issues_system_ids.append(issues_system_id)   

    # Pegar o commit da build considerada como a build de adoção (desconsiderando o primeiro)
    commit_id_DaBuilld = BuildsValidosPorCommit.query('vcs_system_id == @vcs').sort_values(by=['started_at']).iloc[1,0]
    

    # Pegar a release com o commit de build de adoção
    id_tag_Adocao_CD = tagsDeRelase.query('commit_id == @commit_id_DaBuilld').iloc[0,0]
    nome_da_releasa = tagsDeRelase.query('commit_id == @commit_id_DaBuilld').iloc[0,1]
    data_de_adocao = tagsDeRelase.query('commit_id == @commit_id_DaBuilld').iloc[0,6]

    listaDosProjetosComCD.append([name , vcs,project_id, issues_system_id, commit_id_DaBuilld, id_tag_Adocao_CD, nome_da_releasa, data_de_adocao  ])

dfProjetosComCD = pd.DataFrame(listaDosProjetosComCD, columns=['name', 'vcs_system_id', 'project_id', 'issues_system_id', 'commit_id', 'tag_id','adoption_release', 'cd_adoption_date' ])

In [4]:
issuesProjetosCD = issues.query('issue_system_id.isin(@issues_system_ids)')
media_bugs_no_cd = []
mediana_bugs_no_cd = []
media_bugs_cd = []
mediana_bugs_cd = []
for i in range(dfProjetosComCD.shape[0]):
    tmp_name = dfProjetosComCD.iloc[i,0]    
    tmp_date = dfProjetosComCD.iloc[i,7]
    tmp_issue_sistem = dfProjetosComCD.iloc[i,3]
    df_bugs_no_cd = issuesProjetosCD.query('issue_system_id == @tmp_issue_sistem & issue_type == \'Bug\' & status == \'Closed\' & updated_at < @tmp_date')
    df_bugs_no_cd['delta'] = df_bugs_no_cd.updated_at - df_bugs_no_cd.created_at
    media_bugs_no_cd.append(df_bugs_no_cd.delta.mean())
    mediana_bugs_no_cd.append(df_bugs_no_cd.delta.median())

    df_bugs_cd = issuesProjetosCD.query('issue_system_id == @tmp_issue_sistem & issue_type == \'Bug\' &  status == \'Closed\' & created_at >= @tmp_date')
    df_bugs_cd['delta'] = df_bugs_cd.updated_at - df_bugs_cd.created_at
    media_bugs_cd.append(df_bugs_cd.delta.mean())
    mediana_bugs_cd.append(df_bugs_cd.delta.median())

dfProjetosComCD['mean_delta_bugs_no_cd'] = media_bugs_no_cd
dfProjetosComCD['median_delta_bugs_no_cd'] = mediana_bugs_no_cd
dfProjetosComCD['mean_delta_bugs_cd'] = media_bugs_cd
dfProjetosComCD['median_delta_bugs_cd'] = mediana_bugs_cd

C:\Users\diega\AppData\Local\Temp/ipykernel_21140/1288669200.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bugs_no_cd['delta'] = df_bugs_no_cd.updated_at - df_bugs_no_cd.created_at
C:\Users\diega\AppData\Local\Temp/ipykernel_21140/1288669200.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bugs_cd['delta'] = df_bugs_cd.updated_at - df_bugs_cd.created_at
C:\Users\diega\AppData\Local\Temp/ipykernel_21140/1288669200.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [5]:
print('Projetos tempo medio de correção de bugs maior após CD:', dfProjetosComCD.query('mean_delta_bugs_no_cd < mean_delta_bugs_cd').count().iloc[0] )
print('Projetos tempo medio de correção de bugs menor após CD:', dfProjetosComCD.query('mean_delta_bugs_no_cd >= mean_delta_bugs_cd').count().iloc[0] )
print('Projetos com mediana do tempo de correção de bugs maior após CD:', dfProjetosComCD.query('median_delta_bugs_no_cd < median_delta_bugs_cd').count().iloc[0] )
print('Projetos com mediana do tempo de correção de bugs maior após CD:', dfProjetosComCD.query('median_delta_bugs_no_cd >= median_delta_bugs_cd').count().iloc[0] )

Projetos tempo medio de correção de bugs maior após CD: 1
Projetos tempo medio de correção de bugs menor após CD: 19
Projetos com mediana do tempo de correção de bugs maior após CD: 4
Projetos com mediana do tempo de correção de bugs maior após CD: 16


In [6]:
tmpdf = dfProjetosComCD.dropna()

In [7]:
wilcoxon((tmpdf['mean_delta_bugs_no_cd'] / np.timedelta64(1,'D')), (tmpdf['mean_delta_bugs_cd'] / np.timedelta64(1,'D')))

WilcoxonResult(statistic=7.0, pvalue=3.62396240234375e-05)

In [8]:
wilcoxon((tmpdf['mean_delta_bugs_no_cd'] / np.timedelta64(1,'D')), (tmpdf['mean_delta_bugs_cd'] / np.timedelta64(1,'D')), alternative='greater')

WilcoxonResult(statistic=203.0, pvalue=1.811981201171875e-05)

In [9]:
wilcoxon((tmpdf['median_delta_bugs_no_cd'] / np.timedelta64(1,'D')), (tmpdf['median_delta_bugs_cd'] / np.timedelta64(1,'D')))

WilcoxonResult(statistic=21.0, pvalue=0.000850677490234375)

In [10]:
wilcoxon((tmpdf['median_delta_bugs_no_cd'] / np.timedelta64(1,'D')), (tmpdf['median_delta_bugs_cd'] / np.timedelta64(1,'D')), alternative='greater' )

WilcoxonResult(statistic=189.0, pvalue=0.0004253387451171875)

In [12]:
dfProjetosComCD.iloc[0]

name                                             nifi
vcs_system_id                5bd0268235e3ea2b7bbfdbae
project_id                   5bd022e30da3ef1a10738359
issues_system_id             5bdafdb2f16c872096848fba
commit_id                    5bd026e5bad86c1de7dd18ef
tag_id                       5bd026e5bad86c1de7dd18f1
adoption_release                           nifi-1.0.0
cd_adoption_date                  2016-08-31 05:45:12
mean_delta_bugs_no_cd      21 days 11:16:11.030115384
median_delta_bugs_no_cd        6 days 19:06:34.638500
mean_delta_bugs_cd         17 days 15:02:04.997882353
median_delta_bugs_cd           5 days 14:33:31.927000
Name: 0, dtype: object